In [4]:
import pandas as pd

df= pd.read_json('gs://self-embedding/yelp_academic_dataset_review.json', lines=True)

In [ ]:
import sys
!{sys.executable} -m spacy download en

In [ ]:
from __future__ import print_function
import json
import random
from util import Dictionary
from sklearn.utils import shuffle
import spacy

tokenizer = spacy.load('en_core_web_sm')#('en')
dictionary = Dictionary()
dictionary.add_word('<pad>')  # add padding word
with open("tok.json", 'w') as fout:
    df = shuffle(df)
    for i in range(len(df)):
        words = tokenizer(' '.join(df['text'][i].split()))
        data = {
            'label': int(df['stars'][i]) - 1,
            'text': list(map(lambda x: x.text.lower(), words))
        }
        fout.write(json.dumps(data) + '\n')
        for item in data['text']:
            dictionary.add_word(item)
        if i % 100 == 99:
            print('%d/%d files done, dictionary size: %d' %
                    (i + 1, len(df), len(dictionary)))
    fout.close()

with open("dict.json", 'w') as fout:  # save dictionary for fast next process
    fout.write(json.dumps(dictionary.idx2word) + '\n')
    fout.close()

In [ ]:
import pandas as pd
df= pd.read_json('gs://self-embedding/tokens.json', lines=True)

In [5]:
df.head(10)

,label,text
0,1,"[as, someone, who, has, worked, with, many, mu..."
1,0,"[i, am, actually, horrified, this, place, is, ..."
2,4,"[i, love, deagan, 's, ., i, do, ., i, really, ..."
3,0,"[dismal, ,, lukewarm, ,, defrosted, -, tasting..."
4,3,"[oh, happy, day, ,, finally, have, a, canes, n..."
5,4,"[this, is, definitely, my, favorite, fast, foo..."
6,4,"[really, good, place, with, simple, decor, ,, ..."
7,4,"[awesome, office, and, staff, ,, very, profess..."
8,4,"[most, delicious, authentic, italian, i, 've, ..."
9,3,"[i, have, been, here, twice, ., very, nice, an..."


In [10]:
import json
from util import Dictionary
dictionary = Dictionary()
dictionary.add_word('<pad>')  # add padding word
for i in range(len(df)):
    for item in df['text'][i]:
        dictionary.add_word(item)
    if i % 1000 == 99:
        print('%d/%d files done, dictionary size: %d' %
                (i + 1, len(df), len(dictionary)))

with open("dict.json", 'w') as fout:  # save dictionary for fast next process
    fout.write(json.dumps(dictionary.idx2word) + '\n')
    fout.close()

100/1500000 files done, dictionary size: 2358
1100/1500000 files done, dictionary size: 9519
2100/1500000 files done, dictionary size: 13286
3100/1500000 files done, dictionary size: 15882
4100/1500000 files done, dictionary size: 17899
5100/1500000 files done, dictionary size: 19905
6100/1500000 files done, dictionary size: 21735
7100/1500000 files done, dictionary size: 23606
8100/1500000 files done, dictionary size: 25000
9100/1500000 files done, dictionary size: 26651
10100/1500000 files done, dictionary size: 28037
11100/1500000 files done, dictionary size: 29294
12100/1500000 files done, dictionary size: 30484
13100/1500000 files done, dictionary size: 31618
14100/1500000 files done, dictionary size: 32826
15100/1500000 files done, dictionary size: 33944
16100/1500000 files done, dictionary size: 35047
17100/1500000 files done, dictionary size: 36053
18100/1500000 files done, dictionary size: 36977
19100/1500000 files done, dictionary size: 37990
20100/1500000 files done, diction

In [11]:
import numpy as np
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [12]:
train = train.to_json()
with open('train_tok.json', 'w') as outfile:
    json.dump(train, outfile)
validate=validate.to_json()
with open('val_tok.json', 'w') as outfile:
    json.dump(validate, outfile)
test=test.to_json()
with open('test_tok.json', 'w') as outfile:
    json.dump(test, outfile)

In [ ]:
!HYDRA_FULL_ERROR=1 python train.py data.train_data="/home/jupyter/self-embedding/train_tok.json" data.val_data="/home/jupyter/self-embedding/val_tok.json" data.test_data="/home/jupyter/self-embedding/test_tok.json" data.dictionary="/home/jupyter/self-embedding/dict.json" data.word_vector="/home/jupyter/self-embedding/glove.6B.200d.txt.pt" data.save="/home/jupyter/self-embedding/models/model-6B.pt"

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
Begin to load the dictionary.
Loading word vectors from /home/jupyter/self-embedding/glove.6B.200d.txt.pt
115928 words from external word vectors loaded.
Begin to load data.
| epoch   0 |    20/18000 batches | ms/batch 28991.58 | loss 3.6199 | pure loss 1.5501
| epoch   0 |    40/18000 batches | ms/batch 28333.00 | loss 3.2469 | pure loss 1.2778
| epoch   0 |    60/18000 batches | ms/batch 28220.13 | loss 3.1293 | pure loss 1.1638
| epoch   0 |    80/18000 batches | ms/batch 26647.01 | loss 3.1297 | pure loss 1.1737
| epoch   0 |   100/18000 batches | ms/batch 27302.84 | loss 3.0069 | pure loss 1.1022
| epoch   0 |   120/18000 ba